In [22]:
from bs4 import BeautifulSoup
import requests
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import os
from dotenv import load_dotenv

In [23]:
# Load environment variables from .env file
load_dotenv()

# Retrieve the credentials from environment variables
client_id = os.getenv('SPOTIPY_CLIENT_ID')
client_secret = os.getenv('SPOTIPY_CLIENT_SECRET')
redirect_uri = os.getenv('SPOTIPY_REDIRECT_URI')

In [44]:
user_input = input('Which year do you want to travel to? Type the date in this format YYYY: ')

Which year do you want to travel to? Type the date in this format YYYY: 2012


In [45]:
url = f'https://en.wikipedia.org/wiki/Billboard_Year-End_Hot_100_singles_of_{user_input}'
response = requests.get(url)
if response.status_code != 200:
    print(f"Failed to retrieve the page. Status code: {response.status_code}")
soup = BeautifulSoup(response.text, 'lxml')

In [49]:
all_song_titles = soup.select('.wikitable tr td:nth-child(2n)')

In [50]:
title_text = [title.getText().replace('"', '') for title in all_song_titles]

In [52]:
# Define the scope of access
scope = 'playlist-modify-private playlist-read-private'
# Authenticate and create a Spotify object
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=client_id,
                                               client_secret=client_secret,
                                               redirect_uri=redirect_uri,
                                               scope=scope))

In [53]:
# try:
#     # Create a new playlist
#     user_id = sp.current_user()['id']
#     playlist_name = f'Top 100 songs for year {user_input}'
#     playlist_description = f'Top 100 songs for year {user_input}'

#     new_playlist = sp.user_playlist_create(user=user_id,
#                                            name=playlist_name,
#                                            public=False,  # Change to False if you want a private playlist
#                                            description=playlist_description)
#     print(f"Playlist '{new_playlist['name']}' created successfully!")
# except Exception as  e:
#     print(f'Error occur in creating the playlist ', e)
# Retrieve current user's playlists
playlists = sp.current_user_playlists()

In [54]:
track_uri_lst = []
try:
    for track_name in title_text:
        # Construct the query string
        query = f"track:{track_name} year:{user_input}"
        # Search for the track
        results = sp.search(q=query, limit=1, type='track')
        if results['tracks']['items']:
            track_uri_lst.append(results['tracks']['items'][0]['uri'])
        else:
            print(f"No track found for '{query}'")
except Exception as e:
    print(e)

In [56]:
try: 
    playlist_id = playlists['items'][0]['id']
    sp.playlist_add_items(playlist_id, track_uri_lst)
    print("Songs added to the playlist successfully!")
except Exception as e:
    print(e)

Songs added to the playlist successfully!
